<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>


---




<p align="center"><h1 align="center">Quick Start: COVID Tweet Misinformation Text Classification Tutorial</h1> 

##### <p align="center">*Description and dataset adapted from: Sourya Dipta Das, Ayan Basak, and Saikat Dutta. "A Heuristic-driven Ensemble Framework for COVID-19 Fake News Detection”. arXiv preprint arXiv:2101.03545. 2021. Retrieved from https://github.com/diptamath/covid_fake_news/tree/main/data.* 
---

<h3 align="center">(Deploy model to an AI Model Share Model Playground REST API<br> and Web Dashboard in five easy steps...)</h3></p>
<p align="center"><img width="100%" src="https://aimodelsharecontent.s3.amazonaws.com/aimstutorialsteps.gif" /></p>


---



## **Credential Configuration**

In order to deploy an AI Model Share Model Playground, you will need a credentials text file. 

Generating your credentials file requires two sets of information: 
1. Your AI Model Share username and password (create them [HERE](https://www.modelshare.org/login)). 
2. Your AWS (Amazon Web Services) access keys (follow the tutorial [HERE](https://docs.aws.amazon.com/general/latest/gr/aws-sec-cred-types.html)). 

You only need to generate your credentials file once. After running the configure function below, save the outputted file for all your future Model Playground deployments and competition submissions. 

*Note: Handle your credentials file with the same level of security you handle your passwords. Do not share your file with anyone, send via email, or upload to Github.*


In [ ]:
#install aimodelshare library
! pip install aimodelshare --upgrade

In [ ]:
# Generate credentials file
import aimodelshare as ai 
from aimodelshare.aws import configure_credentials 

configure_credentials()

## **Set up Environment**

Use your credentials file to set your credentials for all aimodelshare functions. 

In [2]:
# Set credentials 
import aimodelshare as ai
from aimodelshare.aws import set_credentials

set_credentials(credential_file="credentials.txt", type="deploy_model")

AI Model Share login credentials set successfully.
AWS credentials set successfully.


In [3]:
# Get materials for tutorial: Predicting whether or not a headline is clickbait

import aimodelshare as ai
X_train, X_test, y_train_labels, y_test_labels, example_data = ai.import_quickstart_data("covid_tweets")


Data downloaded successfully.

Preparing downloaded files for use...

Success! Your Quick Start materials have been downloaded. 
You are now ready to run the tutorial.


## **(1) Preprocessor Function & Setup**

### **Write a Preprocessor Function**


> ###   Preprocessor functions are used to preprocess data into the precise data your model requires to generate predictions.  

*  *Preprocessor functions should always be named "preprocessor".*
*  *You can use any Python library in a preprocessor function, but all libraries should be imported inside your preprocessor function.*  

In [4]:
# This preprocessor function makes use of the sklearn TFIDF Vectorizer to create a document term matrix

from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_simple = tf_idf_vectorizer.fit(X_train)

In [5]:
def preprocessor(data):
  from sklearn.feature_extraction.text import TfidfVectorizer
  new_tfidf_df = tfidf_simple.transform(data)
  new_tfidf_df = new_tfidf_df.todense()
  return new_tfidf_df

print(preprocessor(X_train).shape)
print(preprocessor(X_test))

(6505, 18039)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## **(2) Build Model Using your Preferred ML Library**

### **Model 1: Random Forest**

In [6]:
from sklearn.ensemble import RandomForestClassifier

model_1 = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=0)
model_1.fit(preprocessor(X_train), y_train_labels) # Fitting to the training set.
model_1.score(preprocessor(X_test), y_test_labels) # Fit score, 0-1 scale.

0.7610705596107056

In [7]:
#check model predictions
y_pred = model_1.predict(preprocessor(X_test))
y_pred

array(['fake', 'real', 'real', ..., 'real', 'real', 'real'], dtype=object)

## **(3) Save Preprocessor**
### Saves preprocessor function to "preprocessor.zip" file

In [8]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") # Second argument is the directory you want to use to save your function

Your preprocessor is now saved to 'preprocessor.zip'


In [9]:
#  Now let's import and test the preprocessor function to see if it is working...

prep=ai.import_preprocessor("preprocessor.zip")
prep(example_data).shape

(5, 18039)

## **(4) Save sklearn model to ONNX File Format**


In [10]:
# Save sklearn model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

# Check how many preprocessed input features there are
from skl2onnx.common.data_types import FloatTensorType
feature_count=preprocessor(X_test).shape[1]
initial_type = [('float_input', FloatTensorType([None, feature_count]))]  # Insert correct number of preprocessed features

onnx_model = model_to_onnx(model_1, framework='sklearn',
                          initial_types=initial_type,
                          transfer_learning=False,deep_learning=False)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## **(5) Create your Model Playground**

In [11]:
#Set up arguments for Model Playground deployment
import pandas as pd 

model_filepath="model.onnx"
preprocessor_filepath="preprocessor.zip"
exampledata = example_data

In [12]:
# Create one hot encoded data from list of y_train category labels
#...to allow modeltoapi() to extract correct labels for predictions in your deployed API
import pandas as pd
y_train = pd.get_dummies(y_train_labels)

#ensure column names are correct in one hot encoded target for correct label extraction
list(y_train.columns)

['fake', 'real']

In [13]:
from aimodelshare import ModelPlayground

#Instantiate ModelPlayground() Class

myplayground=ModelPlayground(model_type="text", classification=True, private=False)

# Create Model Playground (generates live rest api and web-app for your model/preprocessor)

myplayground.deploy(model_filepath, preprocessor_filepath, y_train, example_data) 

We need some information about your model before we can build your REST API and interactive Model Playground.
   
Model Name (for AI Model Share Website):Covid Misinformation Tweet Classifier
Model Description (Explain what your model does and 
 why end-users would find your model useful):"The significance of social media has increased manifold in the past few decades as it helps people from even the most remote corners of the world stay connected. With the COVID-19 pandemic raging, social media has become more relevant and widely used than ever before, and along with this, there has been a resurgence in the circulation of fake news and tweets that demand immediate attention." Use this dataset to read COVID-19 related tweets and determine if the information they present is "real" or "fake". 
Model Key Words (Search categories that describe your model, separated with commas):covid, tweets, text, binary classification, supervised learning
   
Creating your prediction API. (This process m

## **Use your new Model Playground!**

Follow the link in the output above to:
- Generate predictions with your interactive web dashboard
- Access example code in Python, R, and Curl

Or, follow the rest of the tutorial to create a competition for your Model Playground and: 
- Access verified model performance metrics 
- Upload multiple models to a leaderboard 
- Easily compare model performance & structure 

## **Part 2: Create a Competition**

-------

After deploying your Model Playground, you can now create a competition. 

Creating a competition allows you to:
1. Verify the model performance metrics on aimodelshare.org
2. Submit models to a leaderboard
3. Grant access to other users to submit models to the leaderboard
4. Easily compare model performance and structure 

In [ ]:
# Create list of authorized participants for competition
# Note that participants should use the same email address when creating modelshare.org account

emaillist=["emailaddress1@email.com", "emailaddress2@email.com", "emailaddress3@email.com"]

In [14]:
# Create Competition
# Note -- Make competition public (allow any AI Model Share user to submit models) 
# .... by excluding the email_list argument and including the 'public=True' argument 

myplayground.create_competition(data_directory='covid_tweet_competition_data', 
                                y_test = y_test_labels,  
                              #  email_list = emaillist)
                                public=True)


--INPUT COMPETITION DETAILS--

Enter competition name:Covid Misinformation Competition
Enter competition description:Read tweets about Covid and determine if they represent true information or misinformation.

--INPUT DATA DETAILS--

Note: (optional) Save an optional LICENSE.txt file in your competition data directory to make users aware of any restrictions on data sharing/usage.

Enter data description (i.e.- filenames denoting training and test data, file types, and any subfolders where files are stored):Competition data folder contains labeled X_test, X_train, and y_train_label files.
Enter optional data license descriptive name (e.g.- 'MIT, Apache 2.0, CC0, Other, etc.'):
Uploading your data. Please wait for a confirmation message.

 Success! Model competition created. 

You may now update your prediction API runtime model and verify evaluation metrics with the update_runtime_model() function.

To upload new models and/or preprocessors to this API, team members should use 
the fol

In [15]:
#Instantiate Competition
#--Note: If you start a new session, the first argument should be the Model Playground url in quotes. 
#--e.g.- mycompetition= ai.Competition("https://2121212.execute-api.us-east-1.amazonaws.com/prod/m)
#See Model Playground "Compete" tab for example model submission code.

mycompetition= ai.Competition(myplayground.playground_url)

In [ ]:
# Add, remove, or completely update authorized participants for competition later
emaillist=["emailaddress4@email.com"]

mycompetition.update_access_list(email_list=emaillist,update_type="Add")

Submit Models

In [16]:
#Authorized users can submit new models after setting credentials using modelshare.org username/password
from aimodelshare.aws import set_credentials 

apiurl=myplayground.playground_url # example url from deployed playground: apiurl= "https://123456.execute-api.us-east-1.amazonaws.com/prod/m

set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [17]:
#Submit Model 1: 

#-- Generate predicted values (a list of predicted labels "real" or "fake") (Model 1)
prediction_labels = model_1.predict(preprocessor(X_test))

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 1

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1311


### **Model 2**

In [18]:
# Model 2
from sklearn.ensemble import RandomForestClassifier

model_2 = RandomForestClassifier(n_estimators=300, max_depth=5, random_state=0)
model_2.fit(preprocessor(X_train), y_train_labels) # Fitting to the training set.
model_2.score(preprocessor(X_test), y_test_labels) # Fit score, 0-1 scale.

0.8496350364963504

In [19]:
# Save sklearn model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

# Check how many preprocessed input features there are
from skl2onnx.common.data_types import FloatTensorType
feature_count=preprocessor(X_test).shape[1]
initial_type = [('float_input', FloatTensorType([None, feature_count]))]  # Insert correct number of preprocessed features

onnx_model = model_to_onnx(model_2, framework='sklearn',
                          initial_types=initial_type,
                          transfer_learning=False,deep_learning=False)

with open("model2.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [20]:
# Submit Model 2

#-- Generate predicted values (a list of predicted labels "real" or "fake") (Model 2)
prediction_labels = model_2.predict(preprocessor(X_test))

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model2.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 2

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1311


Get Leaderboard

In [21]:
data = mycompetition.get_leaderboard()
mycompetition.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,model_config,username,version
0,84.96%,84.82%,85.62%,84.73%,sklearn,False,False,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha...",AIModelShare,2
1,76.11%,74.36%,83.13%,75.30%,sklearn,False,False,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha...",AIModelShare,1


Compare Models

In [22]:
# Compare two or more models
data=mycompetition.compare_models([1,2], verbose=1)
mycompetition.stylize_compare(data)

,param_name,default_value,model_version_1,model_version_2
0,bootstrap,True,True,True
1,ccp_alpha,0.000000,0.000000,0.000000
2,class_weight,None,None,None
3,criterion,gini,gini,gini
4,max_depth,None,3,5
5,max_features,auto,auto,auto
6,max_leaf_nodes,None,None,None
7,max_samples,None,None,None
8,min_impurity_decrease,0.000000,0.000000,0.000000
9,min_impurity_split,None,None,None


#### Check structure of y test data 
(This helps users understand how to submit predicted values to leaderboard)

In [23]:
mycompetition.inspect_y_test()

{'class_balance': {'fake': 992, 'real': 1063},
 'class_labels': ['fake', 'real'],
 'label_dtypes': {"<class 'str'>": 2055},
 'y_length': 2055,
 'ytest_example': ['real', 'real', 'fake', 'real', 'real']}

## **Part 3: Maintaining your Model Playground**

-------

Update Runtime model

*Use this function to 1) update the prediction API behind your Model Playground with a new model, chosen from the leaderboard and 2) verify the model performance metrics in your Model Playground*

In [24]:
myplayground.update_runtime_model(model_version=2)

Runtime model & preprocessor for api: https://oxm5jqr4k6.execute-api.us-east-1.amazonaws.com/prod/m updated to model version 2.

Model metrics are now updated and verified for this model playground.


Delete Deployment 

*Use this function to delete the entire Model Playground, including the REST API, web dashboard, competition, and all submitted models*

In [ ]:
myplayground.delete_deployment()

Running this function will permanently delete all resources tied to this deployment, 
 including the eval lambda and all models submitted to the model competition.

To confirm, type 'permanently delete':permanently delete


'API deleted successfully.'